Gleened from https://medium.com/@galen.ballew/transferlearning-b65772083b47



In [ ]:
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing import image

from keras.models import Sequential
import numpy

#import inception with pre-trained weights. do not include fully #connected layers
VGG16_base = VGG16(weights='imagenet', include_top=False)

In [ ]:
x = VGG16_base.output
x = GlobalAveragePooling2D()(x)

# create the full network so we can train on it
VGG16_Bottle = Model(input=VGG16_base.input, output=x)


In [ ]:
import PIL 
import os
from PIL import Image
# Generate dummy data
import numpy as np
import random
#data = np.random.random((1000, 155,200,3))
#labels = np.random.randint(2, size=(1000, numberOfClasses))

#VGG16_transfer.fit(data,labels, epochs=4, batch_size=16)
    
## !! SET THESE PARAMETERS !! ##
NumIm = 3000  # The number of images you want
Its = 0
numD = np.round(NumIm/2)  # Number of daisies


data = np.zeros([NumIm,263,320,3])  # Initialize an array for all the data
outData= np.zeros([NumIm,512])
outLabels=np.zeros([NumIm,2])
   

# Grab some daisies
for file in os.listdir('./daisy'):
    if (Its > numD-1):
        break
    filename = os.fsdecode(file) # Get the file name 
    
    img = image.load_img('./daisy/' + filename, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = VGG16_Bottle.predict(x)
    outData[Its,:]=preds[0,:]
    outLabels[Its,0]=1
    Its += 1
    
    
    

for file in os.listdir('./roses'):
    if (Its > NumIm-1):
        break
    
    filename = os.fsdecode(file) # Get the file name 
    img = image.load_img('./roses/' + filename, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = VGG16_Bottle.predict(x)
    outData[Its,:]=preds[0,:]
    outLabels[Its,1]=1
    Its += 1
    
    

In [ ]:
outData=outData[1:Its-1,:]
outLabels=outLabels[1:Its-1,:]

In [ ]:

np.savetxt("FlowerWeights.csv",outData)
np.savetxt("FlowerLabels.csv",outLabels)

In [ ]:
outData= np.loadtxt("FlowerWeights.csv")
outLabels= np.loadtxt("FlowerLabels.csv")





In [ ]:
myPerm= numpy.random.permutation(outData.shape[0])
trainSize= round(0.8*outData.shape[0])

In [ ]:
outData=outData[myPerm,:]
outLabels=outLabels[myPerm,:]

trainData= outData[:trainSize,:]
trainLabel= outLabels[:trainSize,:]
testData= outData[trainSize:,:]
testLabel= outLabels[trainSize:,:]

In [ ]:
testLabel.shape

In [ ]:
newModel = Sequential()
newModel.add(Dense(256, input_dim=512, activation='relu'))
newModel.add(Dense(256, activation='relu'))
newModel.add(Dense(2, activation='sigmoid'))

newModel.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
newModel.fit(trainData,trainLabel, nb_epoch=150, batch_size=128)

In [ ]:
newModel.evaluate(testData,testLabel)[1]*100

Getting 99% accuracy on the flower data.
Also fast.